In [107]:
import os
from roboflow import Roboflow
from dotenv import load_dotenv

In [108]:
load_dotenv()

True

# **Download Dataset from Roboflow**

In [109]:
ROBOFLOW_API_KEY = os.getenv('ROBOFLOW_API_KEY')

In [110]:
dataset_path = "./datasets"

In [111]:
rf = Roboflow(api_key=ROBOFLOW_API_KEY, model_format="tensorflow")

# this won't download when location folder isn't empty
dataset = rf.workspace().project("capstone-foodlabel").version(5).download(location=dataset_path)

loading Roboflow workspace...
loading Roboflow project...


# **Dataset Inspection**

In [112]:
import pandas as pd

In [113]:
train_dataset_path = dataset_path + "/train"
test_dataset_path = dataset_path + "/test"

In [114]:
df = pd.read_csv(train_dataset_path + "/" + "_annotations.csv", delimiter=",")
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,nutfact-ts-1714374401_png.rf.0001b2093f0dbc09c...,1920,3217,nutrition-label,0,0,1920,3217
1,20240515_162508_png.rf.03397829e6dc1b51443de34...,3468,4624,nutrition-label,445,1856,2332,3384
2,20240515_234932_png.rf.04392d2cfea290d51eea28f...,3468,4624,nutrition-label,665,1890,2327,3979
3,IMG_3559_png.rf.0aa79a4e7078c62542c4856ae5c4e5...,3024,4032,nutrition-label,1012,1624,2409,2850
4,20240516_220241_png.rf.0cab1d59d97c666710799ca...,4624,3468,nutrition-label,685,1865,4222,2714


# **Augment Images**

In [115]:
import cv2
import numpy as np

In [116]:
def to_image_data(filepath):
    return cv2.imread(filepath)

In [117]:
def resize_image(image, width, height):
    resized = cv2.resize(image, (width, height))
    return resized

In [118]:
def to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [119]:
def sharpen_image(image):
    # kernel reference from https://en.wikipedia.org/wiki/Kernel_(image_processing)
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    
    sharpened_image = cv2.filter2D(image, -1, kernel=kernel)
    return sharpened_image

In [120]:
# add 'image' column containing image data
df['image'] = df['filename'].apply(lambda x: to_image_data(train_dataset_path + "/" + x))

# resize image to (size)x(size)
img_size = 32
df['image'] = df['image'].apply(lambda x: resize_image(x, img_size, img_size))

# to grayscale
df['image'] = df['image'].apply(to_grayscale)

# sharpen image
df['image'] = df['image'].apply(sharpen_image)

# **Dataset Cleaning**

In [121]:
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,image
0,nutfact-ts-1714374401_png.rf.0001b2093f0dbc09c...,1920,3217,nutrition-label,0,0,1920,3217,"[[0, 255, 255, 255, 255, 255, 255, 255, 255, 2..."
1,20240515_162508_png.rf.03397829e6dc1b51443de34...,3468,4624,nutrition-label,445,1856,2332,3384,"[[68, 10, 0, 37, 25, 182, 0, 148, 247, 167, 15..."
2,20240515_234932_png.rf.04392d2cfea290d51eea28f...,3468,4624,nutrition-label,665,1890,2327,3979,"[[106, 118, 183, 255, 255, 255, 255, 241, 236,..."
3,IMG_3559_png.rf.0aa79a4e7078c62542c4856ae5c4e5...,3024,4032,nutrition-label,1012,1624,2409,2850,"[[0, 255, 228, 226, 255, 0, 0, 0, 255, 255, 94..."
4,20240516_220241_png.rf.0cab1d59d97c666710799ca...,4624,3468,nutrition-label,685,1865,4222,2714,"[[112, 168, 120, 150, 134, 139, 112, 134, 142,..."


## **Normalize Bounding Boxes**

In [122]:
df['xmin'] = df['xmin'] / df['width']
df['xmax'] = df['xmax'] / df['width']
df['ymin'] = df['ymin'] / df['height']
df['ymax'] = df['ymax'] / df['height']

In [123]:
df['xmin'] = df['xmin'] * img_size
df['xmax'] = df['xmax'] * img_size
df['ymin'] = df['ymin'] * img_size
df['ymax'] = df['ymax'] * img_size

## **Flatten Image Data**

In [124]:
def flatten_image_data(image):
    image_data = np.array(image)
    image_data = image_data.flatten()
    
    return image_data

In [125]:
# df['image'] = df['image'].apply(flatten_image_data)

## **Drop Unused Columns**

In [126]:
df.drop(columns=['filename', 'width', 'height', 'class'], inplace=True)

In [127]:
df.head()

,xmin,ymin,xmax,ymax,image
0,0.000000,0.000000,32.000000,32.000000,"[[0, 255, 255, 255, 255, 255, 255, 255, 255, 2..."
1,4.106113,12.844291,21.517878,23.418685,"[[68, 10, 0, 37, 25, 182, 0, 148, 247, 167, 15..."
2,6.136101,13.079585,21.471742,27.536332,"[[106, 118, 183, 255, 255, 255, 255, 241, 236,..."
3,10.708995,12.888889,25.492063,22.619048,"[[0, 255, 228, 226, 255, 0, 0, 0, 255, 255, 94..."
4,4.740484,17.208766,29.217993,25.042676,"[[112, 168, 120, 150, 134, 139, 112, 134, 142,..."


In [128]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout

In [129]:
model = Sequential()

# convolution
model.add(Conv2D(img_size, (3, 3), input_shape = (img_size,img_size,1), activation = 'relu'))

# pooling
model.add(MaxPooling2D(pool_size = (2, 2)))

# convolution
model.add(Conv2D(img_size, (3, 3),  activation = 'relu'))

# flattening
model.add(Flatten())

# full connection
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 256, activation = 'relu'))

# add Dropout to prevent overfitting
model.add(Dropout(0.5))
model.add(Dense(units = 128, activation = 'relu'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 5408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 512)            │     2,769,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │        32,896 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,943,200 (11.23 MB)

 Trainable params: 2,943,200 (11.23 MB)

 Non-trainable params: 0 (0.00 B)

In [130]:
labels_df = df.drop(columns='image').copy()
labels = tf.constant(([df['xmin'], df['ymin'], df['xmax'], df['ymax']]))
print(tf.shape(labels).numpy())

[  4 170]


In [134]:
dataset = tf.constant((df['image'].to_list()))
print(tf.shape(dataset).numpy())

[170  32  32]
